In [1]:
%%capture
!pip install stable-baselines3[extra]
!pip install moviepy

In [2]:
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback, CallbackList
from stable_baselines3.common.logger import TensorBoardOutputFormat, Video
from stable_baselines3.common.evaluation import evaluate_policy

from typing import Any, Dict

import gymnasium as gym
import torch as th
import numpy as np

CALLBACK_FREQ = 50000
FRAMESKIP = (2, 5)
NUM_TIMESTEPS = 1000000

2024-04-29 15:35:45.830142: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 15:35:45.830237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 15:35:46.003099: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
class VideoRecorderCallback(BaseCallback):
    def __init__(self, eval_env: gym.Env, render_freq: int, n_eval_episodes: int = 1, deterministic: bool = True):
        """
        Records a video of an agent's trajectory traversing ``eval_env`` and logs it to TensorBoard

        :param eval_env: A gym environment from which the trajectory is recorded
        :param render_freq: Render the agent's trajectory every eval_freq call of the callback.
        :param n_eval_episodes: Number of episodes to render
        :param deterministic: Whether to use deterministic or stochastic policy
        """
        super().__init__()
        self._eval_env = eval_env
        self._render_freq = render_freq
        self._n_eval_episodes = n_eval_episodes
        self._deterministic = deterministic

    def _on_step(self) -> bool:
        if self.n_calls % self._render_freq == 0:
            screens = []

            def grab_screens(_locals: Dict[str, Any], _globals: Dict[str, Any]) -> None:
                """
                Renders the environment in its current state, recording the screen in the captured `screens` list

                :param _locals: A dictionary containing all local variables of the callback's scope
                :param _globals: A dictionary containing all global variables of the callback's scope
                """
                screen = self._eval_env.render()
                # PyTorch uses CxHxW vs HxWxC gym (and tensorflow) image convention
                screens.append(screen.transpose(2, 0, 1))

            evaluate_policy(
                self.model,
                self._eval_env,
                callback=grab_screens,
                n_eval_episodes=self._n_eval_episodes,
                deterministic=self._deterministic,
            )
            # Convert screens to a numpy array before passing to pytorch
            screens_array = np.array(screens)
            self.logger.record(
                "trajectory/video",
                Video(th.ByteTensor([screens_array]), fps=20),
                exclude=("stdout", "log", "json", "csv"),
            )
        return True

In [4]:
eval_env = Monitor(gym.make("ALE/Pacman-v5", render_mode="rgb_array", frameskip=FRAMESKIP))
train_env = gym.make("ALE/Pacman-v5", render_mode="rgb_array", frameskip=FRAMESKIP)

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [5]:
eval_callback = EvalCallback(eval_env, log_path="./eval-logs/", eval_freq=CALLBACK_FREQ, n_eval_episodes=5, deterministic=True, render=False)
video_callback = VideoRecorderCallback(eval_env, render_freq=CALLBACK_FREQ)
callback_list = CallbackList([eval_callback, video_callback])

In [6]:
model = DQN(
    "CnnPolicy",
    train_env,
    verbose=1,
    buffer_size=100000,
    tensorboard_log="./tb/")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [7]:
model.learn(total_timesteps=NUM_TIMESTEPS, callback=callback_list, tb_log_name="./control/")

Logging to ./tb/./control/_1


/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/callbacks.py:403: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x7acaa544eda0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7ac984b11990>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 540      |
|    ep_rew_mean      | 12.5     |
|    exploration_rate | 0.979    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 804      |
|    time_elapsed     | 2        |
|    total_timesteps  | 2161     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 519      |
|    ep_rew_mean      | 11.4     |
|    exploration_rate | 0.961    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 792      |
|    time_elapsed     | 5        |
|    total_timesteps  | 4153     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 526      |
|    ep_rew_mean      | 11.7     |
|    exploration_rate | 0.94     |
| time/               |          |
|    episodes       

/opt/conda/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/tmp/ipykernel_25/2676294641.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  Video(th.ByteTensor([screens_array]), fps=20),


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 602      |
|    ep_rew_mean      | 16.5     |
|    exploration_rate | 0.497    |
| time/               |          |
|    episodes         | 88       |
|    fps              | 511      |
|    time_elapsed     | 103      |
|    total_timesteps  | 52962    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000262 |
|    n_updates        | 740      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 606      |
|    ep_rew_mean      | 16.6     |
|    exploration_rate | 0.47     |
| time/               |          |
|    episodes         | 92       |
|    fps              | 458      |
|    time_elapsed     | 121      |
|    total_timesteps  | 55768    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0148   |
|    n_updates      

In [8]:
model.save("ALE-Pacman-v5-control")